In [1]:
import os
import argparse
import numpy as np
import pydicom

 

def load_scan(path):
    # referred from https://www.kaggle.com/gzuidhof/full-preprocessing-tutorial
    sorted_path= os.listdir(path)
    slices = [pydicom.read_file(os.path.join(path, s)) for s in sorted_path]
     
    slices.sort(key=lambda x: float(x.ImagePositionPatient[2]))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
    for s in slices:
        s.SliceThickness = slice_thickness
    return slices


def get_pixels_hu(slices):
    
    image = np.stack([s.pixel_array for s in slices])
    image = image.astype(np.int16)
    image[image == -2000] = 0
    for slice_number in range(len(slices)):
        intercept = slices[slice_number].RescaleIntercept
        slope = slices[slice_number].RescaleSlope
        if slope != 1:
            image[slice_number] = slope * image[slice_number].astype(np.float64)
            image[slice_number] = image[slice_number].astype(np.int16)
        image[slice_number] += np.int16(intercept)
    return np.array(image, dtype=np.int16)

def normalize_(image, MIN_B=-1024.0, MAX_B=3072.0):
   image = (image - MIN_B) / (MAX_B - MIN_B)
   return image

def get_files(path,save_path,data_type):
    id_list= os.listdir(path)
    total_len= 0
    quarter_data = []
    full_data = []
    for id in id_list:
        fd_path= os.path.join(path,id,'full_dose') 
        qd_path= os.path.join(path,id,'quarter_dose') 
 
        full_pixels = get_pixels_hu(load_scan(fd_path))
        quarter_pixels = get_pixels_hu(load_scan(qd_path))
        full_pixels= normalize_(full_pixels)
        quarter_pixels= normalize_(quarter_pixels)
        print(type(full_pixels))
        print(full_pixels.dtype)
        full_data.append(full_pixels)
        quarter_data.append(quarter_pixels)
         
        total_len+=full_pixels.shape[0]
        print(full_pixels.shape)
        print(quarter_pixels.shape)

        print('max : ', np.amax(full_pixels))
        print('min : ', np.amin(full_pixels))
        print('max : ', np.amax(quarter_pixels))
        print('min : ', np.amin(quarter_pixels))
        #print(quarter_pixels.shape)
    
    quarter_data = np.concatenate(quarter_data, axis=0, dtype=np.float32)
    full_data = np.concatenate(full_data, axis=0, dtype=np.float32)
    np.save(os.path.join(save_path, 'x_'+data_type+'.npy'), quarter_data)
    np.save(os.path.join(save_path, 'y_'+data_type+'.npy'), full_data)
    print(total_len)
get_files('dataset/LDCT_raw/train',data_type='train',save_path='dataset/LDCT_npy')

<class 'numpy.ndarray'>
float64
(585, 512, 512)
(585, 512, 512)
max :  0.99853515625
min :  0.0
max :  0.99951171875
min :  0.0
<class 'numpy.ndarray'>
float64
(856, 512, 512)
(856, 512, 512)
max :  0.999755859375
min :  0.0
max :  0.999755859375
min :  0.0
<class 'numpy.ndarray'>
float64
(318, 512, 512)
(318, 512, 512)
max :  0.985595703125
min :  0.0
max :  0.97998046875
min :  0.0
<class 'numpy.ndarray'>
float64
(533, 512, 512)
(533, 512, 512)
max :  0.60595703125
min :  0.0
max :  0.61865234375
min :  0.0
<class 'numpy.ndarray'>
float64
(600, 512, 512)
(600, 512, 512)
max :  0.999755859375
min :  0.0
max :  0.999755859375
min :  0.0
<class 'numpy.ndarray'>
float64
(525, 512, 512)
(525, 512, 512)
max :  0.999755859375
min :  0.0
max :  0.999755859375
min :  0.0
<class 'numpy.ndarray'>
float64
(823, 512, 512)
(823, 512, 512)
max :  0.999755859375
min :  0.0
max :  0.999755859375
min :  0.0
<class 'numpy.ndarray'>
float64
(560, 512, 512)
(560, 512, 512)
max :  0.8212890625
min :  0.0
